In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np
import os

In [5]:
print(tf.__version__)
tf.executing_eagerly()

1.14.0


False

TensorFlow 引入了「Eager Execution」，它是一个命令式、由运行定义的接口，一旦从 Python 被调用，其操作立即被执行。  
这使得入门 TensorFlow 变的更简单，也使研发更直观。  
Eager Execution 的优点如下：  
    快速调试即刻的运行错误并通过 Python 工具进行整合  
    借助易于使用的 Python 控制流支持动态模型  
    为自定义和高阶梯度提供强大支持  
    适用于几乎所有可用的 TensorFlow 运算  

In [ ]:
with tf.Graph().as_default():
    print(tf.executing_eagerly()) # False
    model = Model(num_actions=env.action_space.n)
    agent = A2CAgent(model)
    rewards_history = agent.train(env)
    print("Finished training, testing...")
    print("%d out of 200" % agent.test(env)) # 200 out of 200

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
import gym
env = gym.make('CartPole-v0')
model = Model(env.action_space.n)
model.run_eagerly = True
print("Eager Execution:  ", tf.executing_eagerly())
print("Eager Keras Model:", model.run_eagerly)

ModuleNotFoundError: No module named 'gym'

# Tensorflow主要流程
## 数据结构
### Tensor张量

In [3]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

TensorFlow 支持以下三种类型的张量：
常量：
  常量是其值不能改变的张量。  
  常量存储在计算图的定义中，每次加载图时都会加载相关变量。  
变量：
  当一个量在会话中的值需要更新时，使用变量来表示。
  变量在使用前需要被显示初始化。  
  通过使用变量类来创建。变量的定义还包括应该初始化的常量/随机值。
  例如，在神经网络中，权重需要在训练期间更新，可以通过将权重声明为变量来实现。
占位符：
  用于将值输入TensorFlow图中，可以和feed_dict一起使用来输入数据。  
  在训练神经网络时，它们通常用于提供新的训练样本。
  在会话中运行计算图时，可以为占位符赋值。这样在构建一个计算图时不需要真正地输入数据。
  占位符不包含任何数据，因此不需要初始化它们。
 
 Tensorflow 如果想要从外部传入data, 那就需要用到 tf.placeholder(),   
 然后以这种形式传输数据 sess.run(***, feed_dict={input: **})  

In [19]:
sess = tf.InteractiveSession()
t1 = tf.constant(1)
print(t1.eval())
t2 = tf.constant([[1,2,3,4],[2,3,4,5]])
print(t2.eval())
t3 = tf.constant((2,4,6,8))
print(t3.eval())
t4 = tf.constant(-1.,shape=[3,5],dtype=tf.float32)
print(t4.eval())
sess.close()

#变量
# 注意：变量通常在神经网络中表示权重和偏置。
#权重变量使用正态分布随机初始化，均值为 0，标准差为 2，权重大小为 100×100。偏置由 100 个元素组成，每个元素初始化为 0。
weights = tf.Variable(tf.random_normal(shape=[100,100],stddev=2))
bias = tf.Variable(tf.zeros([100]),name='bias')
#指定一个变量来初始化另一个变量
w2 = tf.Variable(weights.initialized_value(),name='w2')
#变量的定义将指定变量如何被初始化，但是必须显式初始化所有的声明变量
op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(op)
    sess.run(bias)

#每个变量也可以在运行图中单独使用 tf.Variable.initializer来初始化
with tf.Session() as sess:
    sess.run(bias.initializer)
    sess.run(bias)

1
[[1 2 3 4]
 [2 3 4 5]]
[2 4 6 8]
[[-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [20]:
# 启动图会话
sess = tf.Session()
# 定义初始值为0的张量,数据类型默认为float32
tensor1 = tf.zeros(shape=[1,],dtype=tf.int32,name='1dim')
sess.run(tensor1)
tensor2 = tf.zeros(shape=[1,10],dtype=tf.int32,name='2dim')
sess.run(tensor2)
tensor3 = tf.zeros(shape=[3,3],dtype=tf.float32,name='3dim')
sess.run(tensor3)

## 变量和常量
V1 = tf.Variable(tf.zeros([3,3]))
# 不能在运算图上直接执行，需调用initializer执行初始化操作
sess.run(V1.initializer)
sess.run(V1)

row_dim = 2
col_dim = 3
zero_var = tf.Variable(tf.zeros([row_dim, col_dim]))
ones_var = tf.Variable(tf.ones([row_dim, col_dim]))
sess.run(zero_var.initializer)
sess.run(ones_var.initializer)
print(sess.run(zero_var))
print(sess.run(ones_var))

#复制张量,使用ones_like() or zeros_like()
zero_var1 = tf.Variable(tf.zeros_like(zero_var))
sess.run(zero_var1.initializer)
sess.run(zero_var1)

#初始化张量
tensor3 = tf.Variable(tf.fill([2,5],-1.))
sess.run(tensor3.initializer)
sess.run(tensor3)

#创建序列变量
# Linspace in TensorFlow
linear_var = tf.Variable(tf.linspace(start=0.0, stop=2.0, num=6)) # Generates [0.0, 0.5, 1.0] includes the end
# Range in TensorFlow
sequence_var = tf.Variable(tf.range(start=6, limit=15, delta=3)) # Generates [6, 9, 12] doesn't include the end

sess.run(linear_var.initializer)
sess.run(sequence_var.initializer)
print(sess.run(linear_var))
print(sess.run(sequence_var))


#创建随机分布
rnorm_var = tf.random_normal([row_dim, col_dim], mean=0.0, stddev=1.0)
runif_var = tf.random_uniform([row_dim, col_dim], minval=0, maxval=4)
print(sess.run(rnorm_var))
print(sess.run(runif_var))

array([0], dtype=int32)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

[[0. 0. 0.]
 [0. 0. 0.]]
[[1. 1. 1.]
 [1. 1. 1.]]


array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

array([[-1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.]], dtype=float32)

[0.  0.4 0.8 1.2 1.6 2. ]
[ 6  9 12]
[[ 1.1522975   0.13860556  0.15996422]
 [-0.76784575  0.83994615 -0.62967974]]
[[2.2551107  3.8137374  2.6305718 ]
 [0.56810236 2.4097643  1.3143082 ]]


In [21]:
#可视化过程
#执行过程：tensorboard --logdir=./tmp
ops.reset_default_graph()
sess = tf.Session()
my_var = tf.Variable(tf.zeros([5,3]))
merged = tf.summary.merge_all() #添加summary到tensorboard
writer = tf.summary.FileWriter("./tmp/variable_logs", graph=sess.graph)
initialize_op = tf.global_variables_initializer() #初始操作
sess.run(initialize_op) #运行操作

### Placeholder占位符

!tensorboard --logdir=./tmp

In [31]:
holder1 = tf.placeholder(tf.float32,shape=[4,4]) #shape可以是列表或者tuple
#给占位符复制
array = np.random.rand(4,4) #给[4,4数组赋予随机值]
array
y = tf.identity(holder1)
sess.run(y, feed_dict={holder1: array}) #将holder1的值合入到运算图中

array([[0.31513273, 0.54762384, 0.36726584, 0.5515999 ],
       [0.94420841, 0.59701745, 0.1854153 , 0.96239562],
       [0.16329169, 0.16607405, 0.03116262, 0.79993426],
       [0.90795231, 0.18026109, 0.53295571, 0.31128122]])

array([[0.31513274, 0.5476239 , 0.36726585, 0.5515999 ],
       [0.9442084 , 0.59701747, 0.1854153 , 0.9623956 ],
       [0.1632917 , 0.16607405, 0.03116262, 0.79993427],
       [0.9079523 , 0.18026109, 0.5329557 , 0.31128123]], dtype=float32)

In [32]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("/tmp/variable_logs", sess.graph)

In [37]:
# Create data to feed in the placeholder
x_vals = np.array([1., 3., 5., 7., 9.])

# Create the TensorFlow Placceholder
x_data = tf.placeholder(tf.float32)

# Constant for multilication
m = tf.constant(3.)

prod = tf.multiply(x_data,m)

for x in x_vals:
    sess.run(prod,feed_dict={x_data:x})
    
merged = tf.summary.merge_all()
if os.path.exists('tensorboard_logs/'):
    os.makedirs('tensorboard_logs/')
writer = tf.summary.FileWriter('tensorboard_logs/', sess.graph)

3.0

9.0

15.0

21.0

27.0

In [3]:
message = tf.constant('hello tensorflow')
with tf.Session() as sess:
    sess.run(message)
    print(sess.run(message).decode())

b'hello tensorflow'

hello tensorflow


## 程序结构

概念：  
将程序分为两个独立的部分，构建任何拟创建神经网络的蓝图，包括计算图的定义及其执行。  
正是图定义和执行的分开设计让 TensorFlow 能够多平台工作以及并行执行，TensorFlow 也因此更加强大。
 

计算图：是包含节点和边的网络。  
定义数据，也就是张量（tensor）对象（常量、变量和占位符），    
同时定义要执行的所有计算，即运算操作对象（Operation Object，简称 OP）。  

每个节点可以有零个或多个输入，但只有一个输出。  
网络中的节点表示对象（张量和运算操作），边表示运算操作之间流动的张量。 

计算图的执行：   
通过会话执行计算图，部分使用with关键字创建了会话，最后在会话中执行以上计算图。 
会话对象封装了评估张量和操作对象的环境，真正实现了运算操作并将信息从网络的一层传递到另外一层。  
不同张量对象的值仅在会话对象中被初始化、访问和保存。
run(fetches,feed_dict=None,options=None,run_metadata)  

In [10]:
a = tf.constant([1,2,3,4])
b = tf.constant([5,6,7,8])
c = tf.add(a,b)
#with tf.InteractiveSession() as sess:
#在使用Jupyter Notebook或者Python shell进行编程，使用 tf.InteractiveSession将比 tf.Session 更方便。  
#InteractiveSession 使自己成为默认会话，因此你可以使用 eval() 直接调用运行张量对象而不用显式调用会话。
sess = tf.InteractiveSession()
print(c.eval())
a = a + c
print(a.eval())
sess.close()

with tf.Session() as session:
    print(session.run([a,b,c]))

[ 6  8 10 12]
[ 7 10 13 16]
[array([ 7, 10, 13, 16], dtype=int32), array([5, 6, 7, 8], dtype=int32), array([ 6,  8, 10, 12], dtype=int32)]


In [ ]:
'''
cce_obj = metrics.CategoricalCrossentropy()
result = cce_obj(y_true,y_pred)
'''

In [27]:
from keras import metrics
y_true = tf.placeholder(tf.int32, shape=[2,3])
y_pred = tf.placeholder(tf.float32, shape=[2,3])
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = y_true,logits = y_pred)
result = tf.reduce_mean(entropy)
with tf.Session() as sess:
    array1 = np.asarray([[0, 1, 0], [0, 0, 1]])
    array2 = np.asarray([[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
    #y_true和y_pred是占位符；使用它们，在 feed_dict 的帮助下传递包含 X 值的数组和包含 Y 值的数组
    sess.run(result,feed_dict={y_true:array1, y_pred:array2})
    sess.run(entropy,feed_dict={y_true:array1, y_pred:array2})

0.9868951

array([0.5840635, 1.3897266], dtype=float32)